`GB` BigData / [Олег Гладкий](https://gb.ru/users/3837199) // домашнее задание

`262698` __Методы сбора и обработки данных из сети Интернет__:  `04`. __MongoDB__ в Python

# Домашнее задание по теме: MongoDB в Python

1. Развернуть у себя _(на компьютере/виртуальной машине/хостинге)_ базу данных NoSQL __MongoDB__ и реализовать функцию, которая будет добавлять только _новые_ вакансии/продукты в вашу базу.

2. (*) Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы (необходимо анализировать оба поля зарплаты).

3. (*) Любая аналитика. Например matching ваканский с разных площадок



## Задание 0: Читаем вакансии для БД

__Читаем__ данных из файла с диска, полученные в предыдущем дамашнем задании «Парсинг даных: HTML, Beautiful-Soap». Данные получены с сайта `hh.com`. Результаты парсинга в файле `03_hw_HTML_Beautiful-Soap_VACANCIES.json` в `json`-формате.

In [1]:
import json

In [2]:
vacancies_list = list([])

# Читаем результаты парсинга сайта hh.ru (читаем сразу всё, а не частично)

with open('03_hw_HTML_Beautiful-Soap_VACANCIES.json', 'r', encoding='utf-8') as f:
    vacancies_list = json.load(f)
    
# vacancies_list

## Задание 1. MongoDB + вакансии

* Развернуть у себя на компьютере базу данных NoSQL __MongoDB__.
* Реализовать функцию, которая будет добавлять только новые вакансии/продукты в вашу базу.

### Разворачиваем MongoDB

Устанавливаем базу данны MongoDB: сервер, клиент <br>
__Примечание__: при установке необходимо где-то на последнем шаге отказаться от дополнительной установки среды Compass, так как это приводит к очень сильному зависанию всего процесса...

In [3]:
# !mongodb-win32-x86_64-2012plus-4.2.23-signed.msi

Устанавливаем Компас (среду для Mongo) для контроля ситуации из под ОС<br>
__Примечание__: отдельно скачанный установщик отрабатывает предсказуемо быстро.

In [4]:
# !mongodb-compass-1.33.1-win32-x64.exe

Устанавливаем модуль `pymongo` для Питона.<br>
__Примечание__: в нашем случае мы пользуемся средой Anaconda поэтому команда обращена именно к установщику Анаконды

In [5]:
# !conda install pymongo

## Создание БД

Начальная работа с базой данных `MongoDB`:
* Создаём базу, но перед этим удаляем её! Вообще, так как мы решили вопрос дублирования (далее конструкцией try...except), можно базу данных и не удалять. Но для чистоты данного эксперимента и дабы не усложнять задачу — всё же удалим её, подготовив себе чистый плацдарм для работы.
* Инициализируем ссылки на объекты и т. д.

In [6]:
import pandas as pd
from pprint import pprint
import pymongo
import sys

In [7]:
# Подключаемся к серверу баз данных

client = pymongo.MongoClient('mongodb://127.0.0.1:27017') 

# Удаляем базу данных, перед начало работы с ней НА ДАННОМ ЭТАПЕ -- новые данные парсинга отсутствуют
client.drop_database('hh')

# Задаём ссылку на базу данных
db = client.hh

# Создаём ссылку на коллекцию (фактическу таблицу в понимании реляционных БД)
vacancies = db.vacancies

# Проверяем имеющиеся коллекции (таблицы) в БД по нашей ссылке
db.list_collection_names()

[]

### Индекс

#### Дублирование документов: решение проблемы
Создадим уникальный индекс по полю `Link`, включающий `id` вакансии, и исключим дублирование документов БД. В нашем случае этот индекс является дополнительным к уже имеющемуся основному индексу `_id` самой базы данных.

Инфо: https://pymongo.readthedocs.io/en/stable/tutorial.html

In [8]:
# Для этой коллекции создаём индекс (у меня mongo 4.2.23)

index_link = vacancies.create_index([('Link', pymongo.ASCENDING)], unique=True)
index_link, type(index_link)

('Link_1', str)

### Вносим данные в БД

Вносим __только новые__ вакансии: дублирование обрабатываем и отбрасываем.

Данные, полученные ввиде json-файла из предыдущего урока, вносим в базу. 

In [9]:
i_err_dup = 0
i_err_other = 0
i_ok = 0
for i, vacancy in enumerate(vacancies_list, start=1):
    try:
        vacancies.insert_one(vacancy)                   # заносим документы в базу по одному
        
    except pymongo.errors.DuplicateKeyError:
        i_err_dup += 1
    except:
        i_err_other += 1
    else: 
        i_ok += 1

print(f"-----")  # Отчитываемся
print(f"Всего обработано {i} документов: "
      f"успешно {i_ok},",
      f"дублирование {i_err_dup},", 
      f"другие ошибки {i_err_other}.")
print(f"БД (запрос): {vacancies.count_documents({})} документов.")

-----
Всего обработано 1160 документов: успешно 1145, дублирование 15, другие ошибки 0.
БД (запрос): 1145 документов.


## Задание 2

Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы, при этом необходимо следующее:
* необходимо анализировать оба поля зарплаты.

### Задайте уровень дохода!

Необходимо задать:
* уровень дохода `pay_level_ru` в рублях

А так же определить курсы валют: 
* курс доллара `rate_us_dollar`
* курс евро `rate_euro`

Полученная выбрка вакансий будет (в том числе) выведена в файл `04_hw_MongoDB_Python__pay_selection.txt` на диск.

In [10]:
pay_level_ru = 360000       # Задайте зарплату

rate_us_dollar = 60         # Курс доллара 
rate_euro = 55              # Курс евро


# Пересчёт уровня в рубли для "долларовых" вакансий

pay_level_us = pay_level_ru / rate_us_dollar
pay_level_eu = pay_level_ru / rate_euro

curr_ru='руб'
curr_us='USD'
curr_eu='EUR'

Проверяем валюты зарплатных предложений: выводим только уникальные значения поля `Maney_curr`

In [11]:
vacancies.distinct('Maney_curr')

['', 'EUR', 'USD', 'руб.']

#### Формирование запроса

Сформируем запрос для получения выборки (в виде курсора на эти данные). При этом учтём соглашения, принятые для соответствия данным сайта и значениям словаря на примере:
* сайт: от 1000 р.
    * `pay['Maney_min']` = 1000
    * `pay['Maney_max']` = 0
* сайт: от 1000 до 3000 р.
    * `pay['Maney_min']` = 1000
    * `pay['Maney_max']` = 3000 
* сайт: до 3000 р.
    * `pay['Maney_min']` = 0
    * `pay['Maney_max']` = 3000
* сайт: 1000 р.
    * `pay['Maney_min']` = 1000
    * `pay['Maney_max']` = 1000
    
Регулярные выражения: https://www.mongodb.com/docs/manual/reference/operator/query/regex/


In [12]:
arg_ru = [
    {'$and': [{'Maney_min': {'$eq': 0}},               # Рубли
              {'Maney_max': {'$gt': pay_level_ru}},
              {'Maney_curr': {'$regex': curr_ru, '$options': 'i'}}
             ]
    },              

    {'$and': [{'Maney_min': {'$gt': pay_level_ru}}, 
               {'Maney_max': {'$eq': 0}},
               {'Maney_curr': {'$regex': curr_ru, '$options': 'i'}}
              ]
    },

    {'$and': [{'Maney_min': {'$gt': 0}}, 
              {'Maney_max': {'$gt': pay_level_ru}},
              {'Maney_curr': {'$regex': curr_ru, '$options': 'i'}}
             ]
    },
]
arg_us = [
    {'$and': [{'Maney_min': {'$eq': 0}},               # USD
              {'Maney_max': {'$gt': pay_level_us}},
              {'Maney_curr': {'$regex': curr_us, '$options': 'i'}}
             ]
    },              

    {'$and': [{'Maney_min': {'$gt': pay_level_us}}, 
              {'Maney_max': {'$eq': 0}},
              {'Maney_curr': {'$regex': curr_us, '$options': 'i'}}
             ]
    },

    {'$and': [{'Maney_min': {'$gt': 0}}, 
              {'Maney_max': {'$gt': pay_level_us}},
              {'Maney_curr': {'$regex': curr_us, '$options': 'i'}}
             ]
    },
]
arg_eu = [
    {'$and': [{'Maney_min': {'$eq': 0}},               # EUR
              {'Maney_max': {'$gt': pay_level_eu}},
              {'Maney_curr': {'$regex': curr_eu, '$options': 'i'}}
             ]
    },              

    {'$and': [{'Maney_min': {'$gt': pay_level_eu}}, 
              {'Maney_max': {'$eq': 0}},
              {'Maney_curr': {'$regex': curr_eu, '$options': 'i'}}
             ]
    },

    {'$and': [{'Maney_min': {'$gt': 0}}, 
              {'Maney_max': {'$gt': pay_level_eu}},
              {'Maney_curr': {'$regex': curr_eu, '$options': 'i'}}
             ]
    },
]

arg_curr = arg_ru + arg_us + arg_eu
vacancies_selected = vacancies.find({'$or': arg_curr})

#### Выводим результат
Результат выводи на экран в стандартный поток вывода и в файл на диск...

In [13]:
with open('04_hw_MongoDB_Python__pay_selection.txt', 'a', encoding='utf-8') as f_out:
    strims = [sys.stdout, f_out]  # выводим в файл и стандартный поток вывода (для удобства)
    
    for strim in strims:
        print(f"PAY-LEVEL:{pay_level_ru}", file=strim)
        
    for i, vacancy in enumerate(vacancies_selected, start=1):
        for strim in strims:
            print(f"{i:3}  {vacancy['Maney_min']:6}-{vacancy['Maney_max']:6} ",
                  f"{vacancy['Maney_curr']:4}  {vacancy['Name']}", 
                  file=strim)

PAY-LEVEL:360000
  1    6000-  8000  USD   Разработчик C++ (релокация в Dubai)
  2  300000-370000  руб.  Разработчик C++
  3    6500-  6500  USD   Team Lead Python (relocation EU)
  4  400000-450000  руб.  Python Team Lead / Руководитель backend разработки (в аккредитованную ИТ компанию)
  5       0- 10000  EUR   C++ Developer (Berlin)
  6    9500-  9500  USD   Manager, Site Reliability Engineering (to Canada/Serbia)
  7  400000-400000  руб.  DevOps архитектор
  8  400000-400000  руб.  Системный архитектор
  9    7000-  7000  USD   Data Analyst - NLP (to Canada/Serbia)
 10    8000-  8000  USD   Site Reliability Engineer (to Canada/Serbia)
 11    8000-  8000  USD   Senior DevOps Engineer - Automation (to Canada/Serbia)
 12   10000- 10000  USD   Head of DevOps (Dubai)
 13    5500-  6500  USD   Senior Software Engineer (DevOps) – Remote
 14  250000-400000  руб.  Middle/Senior DevOps engineer
 15  250000-450000  руб.  Fullstack техлид (Python)
 16    5500-  7200  USD   Product Owner in QA


<!--  -->

<!--  -->

<!--  -->

__P.S.__



При контроле внесения дубля вакансии не отработана возможность обновления записи, так как дубль может быть более свежей записью, хотя расчитываем, что нам повезло — более свежие копии вакансий должны быть в начале файла данных.

Почему-то вроде-бы такая простая задача получается такой сложной в исполнении?

### Проверка

__Проверено__: 2022-12-27; 18:53
__Оценка__: отлично
__Коммент.__:  все верно выполнено. Можно id сделать как сборный хеш всех полей - тогда проверка на вхождение будет быстрее и эфективнее.
__Ревьювер__: Вадим Мазейко, преподаватель 

<!--  -->

ZIP of code...

In [14]:
%%time
# инициализируем выходную таблицу
# vacancies = pd.DataFrame( \
#     columns=['Name', 'Company', 'Link', 'Date', 'Source', 'Maney_min', 'Maney_max', 'Maney_curr'], \
#     index=[])

CPU times: total: 0 ns
Wall time: 0 ns
